<a href="https://colab.research.google.com/github/KunalSaini03/Mental-Health-Chat-Assistant/blob/main/Personalized_Mental_Health_Chat_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install transformers torch streamlit plotly pandas --quiet
!pip show transformers

Name: transformers
Version: 4.56.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [1]:
!pip install -q kaggle datasets transformers accelerate "torch>=1.13" sentencepiece scikit-learn pandas matplotlib seaborn streamlit pyngrok
import sys, transformers, torch
print("Python", sys.version)
print("Transformers", transformers.__version__)
print("Torch", torch.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.7 MB/s eta 0:00:00
Python 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Transformers 4.56.1
Torch 2.8.0+cu126


In [2]:
import pandas as pd
import os

file_path = "survey.csv"

if not os.path.exists(file_path):
    raise FileNotFoundError(f"File {file_path} not found! Please upload it to /content/ first.")

df = pd.read_csv(file_path, low_memory=False)

print(f"Loaded file: {file_path} | Shape: {df.shape}")
print("Columns available:", df.columns.tolist())
df.head(3)


Loaded file: survey.csv | Shape: (1259, 27)
Columns available: ['Timestamp', 'Age', 'Gender', 'Country', 'state', 'self_employed', 'family_history', 'treatment', 'work_interfere', 'no_employees', 'remote_work', 'tech_company', 'benefits', 'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave', 'mental_health_consequence', 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical', 'obs_consequence', 'comments']


,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN


In [3]:
# Inspect presence of relevant columns
for col in ['comments', 'work_interfere']:
    print(col, "in df:", col in df.columns, "non-null count:", df[col].notna().sum() if col in df.columns else 0)

# Ensure work_interfere exists. If not, try to find equivalent column names.
wi_candidates = [c for c in df.columns if 'work' in c and 'interf' in c]
print("work_interfere candidates:", wi_candidates)
if 'work_interfere' not in df.columns and wi_candidates:
    df = df.rename(columns={wi_candidates[0]: 'work_interfere'})

# Create text column
def make_text_row(row):
    # Prefer free-text comments
    if 'comments' in df.columns and pd.notna(row.get('comments')) and str(row.get('comments')).strip() != "":
        return str(row.get('comments'))
    # Otherwise synthesize a sentence from several available fields (robust)
    pieces = []
    for c in ['self_employed','family_history','treatment','benefits','care_options','anonymity']:
        if c in df.columns:
            v = row.get(c)
            if pd.notna(v):
                pieces.append(f"{c.replace('_',' ')}: {v}")
    # always include work_interfere if present
    if 'work_interfere' in df.columns and pd.notna(row.get('work_interfere')):
        pieces.append(f"work interferes: {row.get('work_interfere')}")
    return ". ".join(pieces) if pieces else ""

df['text'] = df.apply(make_text_row, axis=1)

# Create label from work_interfere
# Mapping: Never/Rarely -> Low (0), Sometimes -> Medium (1), Often -> High (2)
def map_work_interfere_to_label(x):
    if pd.isna(x):
        return None
    x = str(x).strip().lower()
    if x in ['never','rarely','no','none']:
        return 0
    if x in ['sometimes']:
        return 1
    if x in ['often','usually','yes']:
        return 2
    return None

if 'work_interfere' not in df.columns:
    raise ValueError("work_interfere column missing from the dataset. This notebook expects it to be present (Kaggle OSMI datasets should have it).")
df['label'] = df['work_interfere'].apply(map_work_interfere_to_label)

df = df[df['label'].notna() & df['text'].astype(bool)].reset_index(drop=True)
df['label'] = df['label'].astype(int)

print("After filtering, dataset size:", df.shape)
df[['text','work_interfere','label']].head(6)


comments in df: True non-null count: 164
work_interfere in df: True non-null count: 995
work_interfere candidates: ['work_interfere']
After filtering, dataset size: (995, 29)


,text,work_interfere,label
0,family history: No. treatment: Yes. benefits: ...,Often,2
1,family history: No. treatment: No. benefits: D...,Rarely,0
2,family history: No. treatment: No. benefits: N...,Rarely,0
3,family history: Yes. treatment: Yes. benefits:...,Often,2
4,family history: No. treatment: No. benefits: Y...,Never,0
5,family history: Yes. treatment: No. benefits: ...,Sometimes,1


In [4]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)

print("Train:", train_df.shape, "Val:", val_df.shape)
print("Label distribution (train):")
print(train_df['label'].value_counts(normalize=True))
print("Label distribution (val):")
print(val_df['label'].value_counts(normalize=True))


Train: (845, 29) Val: (150, 29)
Label distribution (train):
label
1    0.467456
0    0.388166
2    0.144379
Name: proportion, dtype: float64
Label distribution (val):
label
1    0.466667
0    0.386667
2    0.146667
Name: proportion, dtype: float64


In [5]:
from datasets import Dataset, DatasetDict
train_ds = Dataset.from_pandas(train_df[['text','label']])
val_ds = Dataset.from_pandas(val_df[['text','label']])
dataset = DatasetDict({"train": train_ds, "validation": val_ds})

from transformers import AutoTokenizer
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenize function
def tokenize_fn(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128)

dataset = dataset.map(tokenize_fn, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format(type='torch', columns=['input_ids','attention_mask','labels'])
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/845 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 845
    })
    validation: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 150
    })
})


In [6]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

os.environ["WANDB_DISABLED"] = "true"

# Compute class weights
labels = train_df['label'].to_numpy()
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print("Class weights:", class_weights)

# Model (3 classes)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

from transformers import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/distilbert-stress",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    fp16=False,
    save_total_limit=2,
    seed=42,
    report_to="none"
)

# Metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    prec = precision_score(labels, preds, average='weighted', zero_division=0)
    rec = recall_score(labels, preds, average='weighted', zero_division=0)
    return {"accuracy": acc, "f1": f1, "precision": prec, "recall": rec}


# Initialize Trainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()
trainer.save_model("/content/distilbert-stress/final")
tokenizer.save_pretrained("/content/distilbert-stress/final")
print("Model saved to /content/distilbert-stress/final")

Class weights: tensor([0.8587, 0.7131, 2.3087])


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4020009841.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.514400,0.165739,0.940000,0.939652,0.940352,0.940000
2,0.158800,0.177778,0.940000,0.939652,0.940352,0.940000
3,0.081500,0.226100,0.933333,0.933120,0.933607,0.933333


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Model saved to /content/distilbert-stress/final


In [7]:
# Eval
eval_res = trainer.evaluate()
print("Evaluation results:", eval_res)

# Confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

preds_output = trainer.predict(dataset['validation'])
y_true = preds_output.label_ids
y_pred = np.argmax(preds_output.predictions, axis=1)

print("Classification report:")
print(classification_report(y_true, y_pred, zero_division=0))

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Evaluation results: {'eval_loss': 0.16573946177959442, 'eval_accuracy': 0.94, 'eval_f1': 0.9396516913038653, 'eval_precision': 0.9403523191540495, 'eval_recall': 0.94, 'eval_runtime': 26.6841, 'eval_samples_per_second': 5.621, 'eval_steps_per_second': 0.187, 'epoch': 3.0}


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Classification report:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94        58
           1       0.93      0.97      0.95        70
           2       0.95      0.86      0.90        22

    accuracy                           0.94       150
   macro avg       0.94      0.92      0.93       150
weighted avg       0.94      0.94      0.94       150

Confusion matrix:
 [[54  4  0]
 [ 1 68  1]
 [ 2  1 19]]


In [8]:
from transformers import pipeline
model_path = "/content/distilbert-stress/final"
clf = pipeline("text-classification", model=model_path, tokenizer=model_path, return_all_scores=True, device=0 if torch.cuda.is_available() else -1)

def predict_stress(text):
    # Returns label name and scores
    out = clf(text[:1000])
    # out is list of dicts for each label
    # labels are default 0/1/2; map to named categories
    label_map = {0: "Low", 1: "Medium", 2: "High"}
    # transformer returns list of lists when return_all_scores=True
    scores = {int(d['label'].split('_')[-1]) if 'LABEL_' in d['label'] else i: d['score'] for i,d in enumerate(out[0])}
    predicted = max(scores.items(), key=lambda x: x[1])
    return {"label": label_map[predicted[0]], "scores": {label_map[k]: float(v) for k,v in scores.items()}}

# Example
print(predict_stress("I feel overwhelmed with deadlines and I'm struggling to sleep at night."))


Device set to use cpu


{'label': 'Medium', 'scores': {'Low': 0.36339160799980164, 'Medium': 0.41387882828712463, 'High': 0.22272956371307373}}


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
